In [ ]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime
import sys
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.mixture import BayesianGaussianMixture
import plotly.graph_objects as go
from matplotlib.animation import FuncAnimation

# Add the root project directory to the Python path
project_root = Path.cwd().parent  # This will get the project root since the notebook is in 'notebooks/'
sys.path.append(str(project_root))
from configs.path_config import OUTPUT_DIR

In [ ]:
def load_data(path):
    """
    Description: Load data from a csv file containing strain distributions.

    Args:
        path (path object): The path to the csv file.

    Returns:
        df (pd DataFrame): The data loaded from the csv file.
    """
    df = pd.read_csv(path)
    df.isna().sum().sum()

    return df

# path = OUTPUT_DIR / 'strain_distributions' / 'N-B_Far_Comp_20091129120000_20210611160000_strain_distribution.csv'
path = OUTPUT_DIR / 'strain_distributions' / 'N-F_Mid_Comp_20091129120000_20210611160000_strain_distribution.csv'
df = load_data(path)

df = df.iloc[0:1000,:]

In [ ]:
def remove_outliers(df, threshold=1):
    """
    Description: Remove outliers from the data in two ways:
        1. Remove rows where the mean exceeds the threshold deviation from the overall mean based on absolute values.
        2. Remove rows where any value in the row exceeds the threshold based on its own mean (using absolute values).

    Args:
        df (pd DataFrame): The data loaded from the csv file.
        threshold (float): The threshold for determining outliers based on the mean. Default is 1.

    Returns:
        df_strain (pd DataFrame): The data cleaned of outliers, without the timestamp column.
        df (pd DataFrame): The data cleaned of outliers, with the timestamp column.
    """
    df_strain = df.drop(columns='Timestamp')
    
    # 1. Remove rows where the mean is above the threshold deviation from the overall mean (based on absolute values)
    abs_df = df_strain.abs()  # Take the absolute values of the dataframe
    means = abs_df.mean(axis=1)  # Calculate mean of absolute values for each row
    mean_val = means.mean()  # Mean of all row means
    std_val = means.std()  # Standard deviation of the row means

    # Define a threshold for the mean-based outliers (absolute values)
    mean_threshold = threshold

    # Find outliers based on absolute mean deviation
    mean_outliers = means[np.abs(means - mean_val) > mean_threshold * std_val]
    print("Mean-based outliers (absolute values):")
    print(mean_outliers)

    # 2. Remove rows where any value exceeds the threshold based on the row's mean (using absolute values)
    row_outliers = []
    for idx, row in abs_df.iterrows():
        row_mean = row.mean()  # Mean of the absolute values in the row
        row_std = row.std()  # Standard deviation of the absolute values in the row
        threshold_value = row_mean + threshold * row_std  # Define threshold for each row based on absolute values
        
        # Check if any value in the row exceeds the calculated threshold (absolute value)
        if (row > threshold_value).any():
            row_outliers.append(idx)  # Keep track of the outlier row indices
    
    print("Outliers based on row threshold (absolute values):")
    print(df_strain.loc[row_outliers])

    # Combine both outliers (mean-based and row-based) and drop them
    outliers = mean_outliers.index.union(row_outliers)
    
    # Remove the rows from the data
    df_strain = df_strain.drop(outliers)  # Removed outliers without timestamp
    df = df.drop(outliers)  # Removed outliers with timestamp
    
    print(f"Total number of outliers removed: {len(outliers)}")

    return df_strain, df

# Usage
df_strain, df = remove_outliers(df, threshold=10)

In [ ]:
def do_pca(n_components, df_strain):
    """
    Description: Perform PCA on the data.

    Args:
        n_components (int): The number of principal components to keep.
        df_strain (pd DataFrame): The data.

    Returns:
        df_pca (pd DataFrame): The principal components for each timestamp.
        normalized_pca_components (np array) : The normalized principal components.
    """
    # Perform PCA
    pca = PCA(n_components=n_components)

    # Fit PCA on the entire strain data (matrix-wise)
    pca.fit(df_strain)

    # Apply PCA to the entire strain data (matrix-wise)
    pca_results = pca.transform(df_strain)

    # Normalize the results
    normalized_pca_components = StandardScaler().fit_transform(pca_results)

    # Convert results into a DataFrame
    df_pca = pd.DataFrame(normalized_pca_components, columns=[f'PC{i+1}' for i in range(n_components)])

    # Add timestamps back
    df_pca.insert(0, 'Timestamp', df['Timestamp'].values)

    return normalized_pca_components, df_pca

In [ ]:
import numpy as np
from scipy.stats import multivariate_normal

class DP_FGS:
    def __init__(self, a, l0, R0, j0, m0, omax, max_components=10):
        """
        Initialize the parameters for the Dirichlet Process Gibbs Sampler (DP-FGS).
        
        Args:
            a: Concentration parameter for the Dirichlet Process.
            l0: Hyperparameter for the Gaussian mean (prior mean).
            R0: Hyperparameter for the Gaussian covariance (prior covariance).
            j0: Hyperparameter for the cluster size.
            m0: Hyperparameter for the new cluster's mean.
            omax: Maximum number of recent data points considered for updating.
            max_components: Maximum number of clusters to consider.
        """
        self.a = a
        self.l0 = l0
        self.R0 = R0
        self.j0 = j0
        self.m0 = m0
        self.omax = omax
        self.max_components = max_components
        
        self.cluster_params = []
        self.cluster_labels = []
        self.data_points = []
        
    def _initialize_cluster_params(self, data_point):
        """
        Initialize cluster parameters for a new cluster.
        
        Args:
            data_point: New data point.
        
        Returns:
            tuple: Initialized mean and covariance for the new cluster.
        """
        mean = np.random.normal(self.l0, self.m0)
        covariance = self.R0 * np.eye(len(data_point))  # Assuming diagonal covariance matrix
        return mean, covariance
        
    def _predict_cluster(self, data_point, cluster_id):
        """
        Predict the likelihood of the data point for a given cluster using a Gaussian distribution.
        
        Args:
            data_point: New data point.
            cluster_id: Cluster index to evaluate.
        
        Returns:
            float: Log-likelihood of the data point under the Gaussian distribution of the cluster.
        """
        mean, covariance = self.cluster_params[cluster_id]
        
        # Ensure data_point is a 1D array
        data_point = np.squeeze(data_point)  # Flatten the data point if it's not already a 1D array
        
        if mean.ndim != 1:
            raise ValueError(f"Mean should be a 1D array, but got shape {mean.shape}")
        if data_point.ndim != 1:
            raise ValueError(f"Data point should be a 1D array, but got shape {data_point.shape}")
        
        # Now compute the log-likelihood
        return multivariate_normal.logpdf(data_point, mean=mean, cov=covariance)
        
    def _sample_cluster_assignment(self, data_point):
        """
        Sample the cluster assignment for the new data point based on the predictive posterior.
        
        Args:
            data_point: New data point.
        
        Returns:
            int: Cluster index that the data point is assigned to.
        """
        log_likelihoods = []
        for cluster_id in range(len(self.cluster_params)):
            log_likelihoods.append(self._predict_cluster(data_point, cluster_id))
        
        # For new cluster
        log_likelihoods.append(np.log(self.a))  # Prior for a new cluster
        
        # Compute normalized probabilities (using softmax)
        log_probs = np.array(log_likelihoods) - np.max(log_likelihoods)  # Avoid numerical issues
        probs = np.exp(log_probs) / np.sum(np.exp(log_probs))
        
        return np.random.choice(len(self.cluster_params) + 1, p=probs)
        
    def _update_cluster_params(self, cluster_id, data_point):
        """
        Update the parameters (mean and covariance) of a cluster based on the assigned data point.
        
        Args:
            cluster_id: The cluster to update.
            data_point: New data point assigned to this cluster.
        """
        # Use conjugate priors to update mean and covariance (simplified version)
        mean, covariance = self.cluster_params[cluster_id]
        
        # Here, you would implement the actual parameter update using the data and the prior.
        # This is a placeholder, assuming updates based on a simple Gaussian update rule.
        
        updated_mean = np.mean(data_point)  # Update with new data
        updated_cov = covariance + np.var(data_point)  # Placeholder update rule
        
        self.cluster_params[cluster_id] = (updated_mean, updated_cov)
    
    def fit(self, data):
        """
        Perform dynamic clustering on the data using the Gibbs sampler.
        
        Args:
            data: The dataset (2D array) where each row is a data point to be clustered.
        """
        N = 0  # Number of observed points
        for i, data_point in enumerate(data):
            N += 1
            if N > self.omax:
                o = self.omax
            else:
                o = N  # Number of points to consider
            
            # Random permutation of the last 'o' data points
            recent_data_points = self.data_points[-o:]
            np.random.shuffle(recent_data_points)
            
            # Sample a cluster assignment for the new point
            cluster_id = self._sample_cluster_assignment(data_point)
            
            if cluster_id == len(self.cluster_params):  # New cluster
                mean, covariance = self._initialize_cluster_params(data_point)
                self.cluster_params.append((mean, covariance))
            
            # Add the point to the assigned cluster
            self.cluster_labels.append(cluster_id)
            self.data_points.append(data_point)
            
            # Update the cluster parameters
            self._update_cluster_params(cluster_id, data_point)
    
    def get_cluster_assignments(self):
        """Get the cluster assignments for all data points."""
        return self.cluster_labels


# Example Usage:

# Initialize the DP-FGS model
dp_fgs = DP_FGS(a=1.0, l0=0.0, R0=1.0, j0=1.0, m0=0.0, omax=10)

# Example data points (replace with actual SHM data)
data = np.random.randn(100, 2)  # 100 data points, 2 features

# Perform dynamic clustering
dp_fgs.fit(data)

# Get cluster assignments
cluster_assignments = dp_fgs.get_cluster_assignments()

# Print cluster assignments for each data point
print(cluster_assignments)



In [ ]:
def dynamic_clustering_dpgmm(data, max_components=10, init_points=10):
    """
    Perform dynamic clustering on the data, adding one data point at a time, using DPGMM.
    
    Args:
        data (np.ndarray): The dataset to cluster (PCA components).
                          This should be a 2D numpy array where each row represents 
                          the PCA components of a strain distribution at a particular timestamp.
                          Shape: (n_samples, n_features)
        max_components (int): Maximum number of components for Bayesian GMM.
        init_points (int): Number of initial points to use for model initialization.

    Returns:
        pd.DataFrame: A DataFrame containing the original PCA components and their corresponding cluster labels.
                      Shape: (n_samples, n_features + 1) where the last column is 'Cluster'.
    """
    # Initialize the model with the maximum components and full covariance
    model = BayesianGaussianMixture(n_components=max_components, covariance_type='full', random_state=42)
    
    # Initialize the list to store cluster labels for each point
    cluster_labels = []
    
    # First fit the model with the first 'init_points' points
    initial_data = data[:init_points]
    
    # Instead of predicting the clusters, manually assign cluster 0 to all initial points
    cluster_labels.extend([0] * init_points)
    
    # Fit the model with the initial data points
    model.fit(initial_data)
    
    # For the first 'init_points', assign clusters
    # cluster_labels.extend(model.predict(initial_data))  # Predict cluster labels for initial points
    
    # Then, fit and predict incrementally for the remaining points
    for i in range(init_points, len(data)):
        point = data[i].reshape(1, -1)  # Reshape the data to be a 2D array (one row)

        # Fit the model with data up to the current point (i+1)
        model.fit(data[:i+1])  # Incrementally fit the model with all data up to this point
        
        # Predict the cluster for the current point
        cluster_label = model.predict(point)
        cluster_labels.append(cluster_label[0])
        # print(f'Cluster labels: \n{cluster_labels}')
    
    # Create a DataFrame with the original PCA components and cluster labels
    data_with_dpgmm = pd.DataFrame(data, columns=[f'PC{i+1}' for i in range(data.shape[1])])
    data_with_dpgmm['Cluster'] = cluster_labels  # Add the cluster labels as a new column
    
    return data_with_dpgmm

def plot_dynamic_clustering(data_with_dpgmm):
    """
    Plot the results of dynamic clustering over time, showing how cluster assignments evolve.
    
    Args:
        data_with_dpgmm (pd.DataFrame): DataFrame containing PCA components and their corresponding cluster labels.
                                         It should have columns 'PC1', 'PC2', ..., 'Cluster'.
    """
    # Extract the first two principal components for visualization
    pc1 = data_with_dpgmm['PC1']
    pc2 = data_with_dpgmm['PC2']
    
    # Create a scatter plot where the color represents the cluster label
    plt.figure(figsize=(10, 6))
    scatter = plt.scatter(pc1, pc2, c=data_with_dpgmm['Cluster'], cmap='viridis', s=50)
    
    # Add color bar to indicate the cluster labels
    plt.colorbar(scatter, label='Cluster')
    
    # Add labels and title
    plt.xlabel('PC1')
    plt.ylabel('PC2')
    plt.title('Dynamic Clustering of PCA Components')
    
    # Show the plot
    plt.show()

# Example usage:
n_components = 10
# df_strain should have shape (n_samples, n_features), where n_samples is the number of timestamps, 
# and n_features is the number of features (or sensors).
normalized_pca_components, df_pca = do_pca(n_components, df_strain)  # Perform PCA

# Perform dynamic clustering using DPGMM
data_with_dpgmm = dynamic_clustering_dpgmm(normalized_pca_components, max_components=10, init_points=10)
plot_dynamic_clustering(data_with_dpgmm)

In [ ]:
data_with_dpgmm

In [ ]:
def dpgmm_clustering(normalized_pca_components, df):

    bgm = BayesianGaussianMixture(n_components=10, covariance_type='full', random_state=42)
    bgm.fit(normalized_pca_components)
    clusters = bgm.predict(normalized_pca_components)

    # Add cluster labels to your original data (without overwriting)
    data_with_dpgmm = df.copy()  # Make a copy to preserve the original DataFrame

    # Insert the clusters as the second column (at index 1)
    data_with_dpgmm.insert(1, 'Cluster', clusters)

    n_clusters = len(data_with_dpgmm['Cluster'].unique())

    # data_with_dpgmm.insert(2, 'Assigned_Cluster_Prob', assigned_prob)  # Insert probability for the assigned cluster


    # Count the number of data points assigned to each cluster
    cluster_counts = {i: sum(clusters == i) for i in range(n_clusters)}

    # Plot the clusters
    plt.figure(figsize=(10, 6))
    scatter = sns.scatterplot(x=normalized_pca_components[:, 0], y=normalized_pca_components[:, 1], hue=clusters, palette="viridis", s=100, alpha=0.7)

    # Create custom labels for the legend with the cluster counts
    legend_labels = [f'Cluster {i} (n={cluster_counts[i]})' for i in range(n_clusters)]
    handles, _ = scatter.get_legend_handles_labels()

    # Set the custom labels in the legend
    plt.legend(handles=handles, labels=legend_labels, title='Cluster')

    # Label the axes
    plt.xlabel("Principal Component 1")
    plt.ylabel("Principal Component 2")
    plt.title("PCA + dpgmm Clustering")

    # Show the plot
    plt.show()

    # Show the updated DataFrame with the Cluster column as the second column
    return data_with_dpgmm

n_components = 10
normalized_pca_components, df_pca = do_pca(n_components, df_strain)

data_with_dpgmm = dpgmm_clustering(normalized_pca_components, df)
# data_with_dpgmm

In [ ]:
def plot_clusters_over_time(data_with_clusters, method) -> None:
    """
    Description: Plot the assignment to clusters over time with any clustering method.

    Args:
        data_with_KMeans (pd DataFrame): The original DataFrame including the timestamps with the addition of the cluster labels.
        metod (str): The clustering method used (e.g., 'KMeans', 'dpgmm', 'DBSCAN'). This will be used in the title.

    Returns:
        None
    """
    # Assuming 'Timestamp' is a column with string dates, we convert it to datetime format.
    data_with_clusters['Timestamp'] = pd.to_datetime(df['Timestamp'])

    # Create the Plotly figure
    fig = go.Figure()

    # Add a scatter plot (you can choose 'line' or 'scatter' depending on the style you want)
    fig.add_trace(go.Scatter(x=data_with_clusters['Timestamp'], 
                            y=data_with_clusters['Cluster'], 
                            mode='markers+lines',  # markers and lines
                            marker=dict(size=8, 
                                        color=data_with_clusters['Cluster'],  # Color by cluster value
                                        colorscale='Viridis',  # You can change the colorscale here
                                        colorbar=dict(title='Cluster')),  # Add a color bar to show the scale
                            line=dict(width=1, color='grey')))  # Customizing line color and width

    # Update layout with title and labels
    fig.update_layout(
        title=f'Assignment to Clusters Over Time with {method} Clustering',
        xaxis_title='Time',
        yaxis_title='Cluster',
        xaxis_tickangle=-45,  # Rotate x-axis labels for better readability
        yaxis=dict(tickmode='linear', tick0=0, dtick=1)  # Set y-tick step size to 1
    )

    # Show the figure
    fig.show()

plot_clusters_over_time(data_with_dpgmm, 'DPGMM')